In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Flatten, Dropout, Reshape, MultiHeadAttention, Conv2D, Conv3D, DepthwiseConv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, GlobalAveragePooling3D, Multiply, Input
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def extract_frames(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // frame_count

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (64, 64))
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

def load_data(data_dir):
    X = []
    y = []
    labels = {'Defensive': 0, 'Pull': 1, 'Drive': 2, 'Flick': 3}

    for label in labels:
        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)
        for video in videos:
            video_path = os.path.join(videos_dir, video)
            frames = extract_frames(video_path)
            if len(frames) == 10:
                X.append(frames)
                y.append(labels[label])

    X = np.array(X)
    y = np.array(y)
    return X, y

def check_data(data_dir, model):
    labels = {'Defensive': 0, 'Pull': 1, 'Drive': 2, 'Flick': 3}

    analysis = {}

    for label in labels:
        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)

        analysis[label] = {
            'OK': [],
            'NOT OK': []
        }

        for video in videos:
            video_path = os.path.join(videos_dir, video)

            frames = extract_frames(video_path)
            frames = np.expand_dims(frames, axis=0)
            frames = np.expand_dims(frames, axis=-1)

            prediction = model.predict(frames)
            predicted_class = np.argmax(prediction, axis=1)

            if predicted_class[0] == labels[label]:
                analysis[label]['OK'].append(video.split('.')[0])
            else:
                analysis[label]['NOT OK'].append(video.split('.')[0])

    return analysis

In [ ]:
data_dir = '/content/drive/My Drive/CroppedDetection'
X, y = load_data(data_dir)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0

In [ ]:
# (batch_size, time_steps, height, width, channels)
X_train = np.reshape(X_train, (X_train.shape[0], 10, 64, 64, 3))
X_val = np.reshape(X_val, (X_val.shape[0], 10, 64, 64, 3))

In [ ]:
input_shape = X_train.shape[1:]
input_shape

(10, 64, 64, 3)

In [ ]:
# def squeeze_excite_block(inputs, ratio=16):
#     filters = inputs.shape[-1]
#     se = GlobalAveragePooling2D()(inputs)
#     se = Reshape((1, 1, filters))(se)
#     se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
#     se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
#     return Multiply()([inputs, se])

# def mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio, se_ratio=0):
#     channel_axis = -1
#     input_filters = inputs.shape[channel_axis]
#     x = Conv2D(expand_ratio * input_filters, 1, padding='same', use_bias=False)(inputs)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('relu')(x)

#     x = DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=False)(x)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('relu')(x)

#     if se_ratio:
#         x = squeeze_excite_block(x, ratio=int(1/se_ratio))

#     x = Conv2D(filters, 1, padding='same', use_bias=False)(x)
#     x = BatchNormalization(axis=channel_axis)(x)

#     if strides == 1 and input_filters == filters:
#         x = Add()([x, inputs])

#     return x

# def fused_mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio):
#     channel_axis = -1
#     input_filters = inputs.shape[channel_axis]
#     x = Conv2D(expand_ratio * input_filters, kernel_size, strides=strides, padding='same', use_bias=False)(inputs)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('relu')(x)

#     x = Conv2D(filters, 1, padding='same', use_bias=False)(x)
#     x = BatchNormalization(axis=channel_axis)(x)

#     if strides == 1 and input_filters == filters:
#         x = Add()([x, inputs])

#     return x

# def create_efficient_net_v2s(input_shape):
#     inputs = Input(shape=input_shape)

#     # Stage 0
#     x = Conv2D(24, (3, 3), strides=(2, 2), padding='same', use_bias=False)(inputs)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     print(f'Stage 0 output shape: {x.shape}')

#     # Stage 1
#     for _ in range(2):
#         x = fused_mb_conv_block(x, 24, 3, 1, 1)
#     print(f'Stage 1 output shape: {x.shape}')

#     # Stage 2
#     for _ in range(4):
#         x = fused_mb_conv_block(x, 48, 3, 2, 4)
#     print(f'Stage 2 output shape: {x.shape}')

#     # Stage 3
#     for _ in range(4):
#         x = fused_mb_conv_block(x, 64, 3, 2, 4)
#     print(f'Stage 3 output shape: {x.shape}')

#     # Stage 4
#     for _ in range(6):
#         x = mb_conv_block(x, 128, 3, 2, 4, 0.25)
#     print(f'Stage 4 output shape: {x.shape}')

#     # Stage 5
#     for _ in range(9):
#         x = mb_conv_block(x, 160, 3, 1, 6, 0.25)
#     print(f'Stage 5 output shape: {x.shape}')

#     # Stage 6
#     for _ in range(15):
#         x = mb_conv_block(x, 256, 3, 2, 6, 0.25)
#     print(f'Stage 6 output shape: {x.shape}')

#     x = GlobalAveragePooling2D()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dense(128, activation='relu')(x)
#     outputs = Dense(10, activation='softmax')(x)  # Assuming 10 action classes

#     model = Model(inputs, outputs)
#     return model

In [ ]:
# def squeeze_excite_block(inputs, ratio=16):
#     filters = inputs.shape[-1]
#     se = GlobalAveragePooling3D()(inputs)
#     se = Reshape((1, 1, 1, filters))(se)
#     se = Dense(filters // ratio, activation='tanh', kernel_initializer='he_normal', use_bias=False)(se)
#     se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
#     return Multiply()([inputs, se])

# def mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio, se_ratio=0):
#     channel_axis = -1
#     input_filters = inputs.shape[channel_axis]
#     x = Conv3D(expand_ratio * input_filters, (1, 1, 1), padding='same', use_bias=False)(inputs)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('tanh')(x)

#     x = Conv3D(expand_ratio * input_filters, kernel_size, strides=strides, padding='same', use_bias=False, groups=expand_ratio * input_filters)(x)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('tanh')(x)

#     if se_ratio:
#         x = squeeze_excite_block(x, ratio=int(1/se_ratio))

#     x = Conv3D(filters, (1, 1, 1), padding='same', use_bias=False)(x)
#     x = BatchNormalization(axis=channel_axis)(x)

#     if strides == 1 and input_filters == filters:
#         x = Add()([x, inputs])

#     return x

# def fused_mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio):
#     channel_axis = -1
#     input_filters = inputs.shape[channel_axis]
#     x = Conv3D(expand_ratio * input_filters, kernel_size, strides=strides, padding='same', use_bias=False)(inputs)
#     x = BatchNormalization(axis=channel_axis)(x)
#     x = Activation('tanh')(x)

#     x = Conv3D(filters, (1, 1, 1), padding='same', use_bias=False)(x)
#     x = BatchNormalization(axis=channel_axis)(x)

#     if strides == 1 and input_filters == filters:
#         x = Add()([x, inputs])

#     return x

# def build_efficient_net_v2s_conv_core(input_shape):
#     inputs = Input(shape=input_shape)

#     # Stage 0
#     x = Conv3D(24, (3, 3, 3), strides=(1, 2, 2), padding='same', use_bias=False)(inputs)
#     x = BatchNormalization()(x)
#     x = Activation('tanh')(x)
#     print(f'Stage 0 output shape: {x.shape}')

#     # Stage 1
#     for _ in range(2):
#         x = fused_mb_conv_block(x, 24, (3, 3, 3), 1, 1)
#     print(f'Stage 1 output shape: {x.shape}')

#     # Stage 2
#     for _ in range(4):
#         x = fused_mb_conv_block(x, 48, (3, 3, 3), (1, 2, 2), 4)
#     print(f'Stage 2 output shape: {x.shape}')

#     # Stage 3
#     for _ in range(4):
#         x = fused_mb_conv_block(x, 64, (3, 3, 3), (1, 2, 2), 4)
#     print(f'Stage 3 output shape: {x.shape}')

#     # Stage 4
#     for _ in range(6):
#         x = mb_conv_block(x, 128, (3, 3, 3), (1, 2, 2), 4, 0.25)
#     print(f'Stage 4 output shape: {x.shape}')

#     # Stage 5
#     for _ in range(9):
#         x = mb_conv_block(x, 160, (3, 3, 3), 1, 6, 0.25)
#     print(f'Stage 5 output shape: {x.shape}')

#     # Stage 6
#     for _ in range(15):
#         x = mb_conv_block(x, 256, (3, 3, 3), (1, 2, 2), 6, 0.25)
#     print(f'Stage 6 output shape: {x.shape}')

#     x = GlobalAveragePooling3D()(x)
#     print(f'Averaging Stage output shape: {x.shape}')

#     x = Dense(256, activation='tanh')(x)
#     print(f'FNN 0 output shape: {x.shape}')

#     x = Dense(128, activation='tanh')(x)
#     print(f'FNN 1 output shape: {x.shape}')

#     outputs = Dense(10, activation='softmax')(x)  # Assuming 10 action classes
#     print(f'Final output shape: {outputs.shape}')

#     model = Model(inputs, outputs)
#     return model

In [ ]:
def squeeze_excite_block(inputs, ratio=16):
    filters = inputs.shape[-1]
    se = GlobalAveragePooling3D()(inputs)
    se = Reshape((1, 1, 1, filters))(se)
    se = Dense(filters // ratio, activation='tanh', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    return Multiply()([inputs, se])

def mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio, se_ratio=0):
    channel_axis = -1
    input_filters = inputs.shape[channel_axis]
    x = Conv3D(expand_ratio * input_filters, (1, 1, 1), padding='same', use_bias=False)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('tanh')(x)

    x = Conv3D(expand_ratio * input_filters, kernel_size, strides=strides, padding='same', use_bias=False, groups=expand_ratio * input_filters)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('tanh')(x)

    if se_ratio:
        x = squeeze_excite_block(x, ratio=int(1/se_ratio))

    x = Conv3D(filters, (1, 1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if strides == 1 and input_filters == filters:
        x = Add()([x, inputs])

    return x

def fused_mb_conv_block(inputs, filters, kernel_size, strides, expand_ratio):
    channel_axis = -1
    input_filters = inputs.shape[channel_axis]
    x = Conv3D(expand_ratio * input_filters, kernel_size, strides=strides, padding='same', use_bias=False)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('tanh')(x)

    x = Conv3D(filters, (1, 1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if strides == 1 and input_filters == filters:
        x = Add()([x, inputs])

    return x

def efficient_net_v2s_conv_core(input_shape):
    inputs = Input(shape=input_shape)

    # Stage 0
    x = Conv3D(24, (3, 3, 3), strides=(1, 2, 2), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('tanh')(x)
    print(f'Stage 0 output shape: {x.shape}')

    # Stage 1
    for _ in range(2):
        x = fused_mb_conv_block(x, 24, (3, 3, 3), 1, 1)
    print(f'Stage 1 output shape: {x.shape}')

    # Stage 2
    for _ in range(4):
        x = fused_mb_conv_block(x, 48, (3, 3, 3), (1, 2, 2), 4)
    print(f'Stage 2 output shape: {x.shape}')

    # Stage 3
    for _ in range(4):
        x = fused_mb_conv_block(x, 64, (3, 3, 3), (1, 2, 2), 4)
    print(f'Stage 3 output shape: {x.shape}')

    # Stage 4
    for _ in range(6):
        x = mb_conv_block(x, 128, (3, 3, 3), (1, 2, 2), 4, 0.25)
    print(f'Stage 4 output shape: {x.shape}')

    # Stage 5
    for _ in range(9):
        x = mb_conv_block(x, 160, (3, 3, 3), 1, 6, 0.25)
    print(f'Stage 5 output shape: {x.shape}')

    # Stage 6
    for _ in range(15):
        x = mb_conv_block(x, 256, (3, 3, 3), (1, 2, 2), 6, 0.25)
    print(f'Stage 6 output shape: {x.shape}')

    outputs = x

    model = Model(inputs, outputs)
    return model

In [ ]:
def create_LRCN_effNet_model(input_shape, num_classes):
    model = Sequential()

    # Layer 1
    model.add(TimeDistributed(efficientnet, input_shape=input_shape))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 2
    model.add(TimeDistributed(efficientnet))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 3
    model.add(TimeDistributed(efficientnet))
    model.add(TimeDistributed(Dropout(0.2)))

    # Flatten the output
    model.add(TimeDistributed(Flatten()))

    # LSTM Layer
    model.add(LSTM(32))

    # Reshape for Multi-Head Attention
    model.add(Reshape((1, 1280)))
    model.add(MultiHeadAttention(num_heads=8, key_dim=128, name='attention_layer'))
    model.add(Reshape((512,)))

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
num_classes = 4

In [ ]:
model = create_LRCN_effNet_model(input_shape, num_classes)
model.summary()

ValueError: Exception encountered when calling layer "time_distributed_5" (type TimeDistributed).

Input 0 of layer "efficientnetv2-b0" is incompatible with the layer: expected shape=(None, 64, 64, 3), found shape=(None, 1280)

Call arguments received by layer "time_distributed_5" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 10, 1280), dtype=float32)
  • training=None
  • mask=None

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/30
64/64 [==============================] - 33s 476ms/step - loss: 1.3918 - accuracy: 0.2598 - val_loss: 1.3828 - val_accuracy: 0.2607
Epoch 2/30
64/64 [==============================] - 28s 442ms/step - loss: 1.3752 - accuracy: 0.2812 - val_loss: 1.3970 - val_accuracy: 0.2996
Epoch 3/30
64/64 [==============================] - 29s 459ms/step - loss: 1.3727 - accuracy: 0.3008 - val_loss: 1.3532 - val_accuracy: 0.3541
Epoch 4/30
64/64 [==============================] - 29s 452ms/step - loss: 1.3385 - accuracy: 0.3633 - val_loss: 1.2963 - val_accuracy: 0.4436
Epoch 5/30
64/64 [==============================] - 26s 410ms/step - loss: 1.3295 - accuracy: 0.3564 - val_loss: 1.2455 - val_accuracy: 0.4514
Epoch 6/30
64/64 [==============================] - 28s 432ms/step - loss: 1.2308 - accuracy: 0.4346 - val_loss: 1.1492 - val_accuracy: 0.5214
Epoch 7/30
64/64 [==============================] - 28s 443ms/step - loss: 1.1035 - accuracy: 0.5371 - val_loss: 1.0170 - val_accuracy: 0.6109

In [ ]:
model.save('/content/drive/My Drive/CroppedDetection/effNet_model.h5')

NameError: name 'model' is not defined

In [ ]:
model_path = '/content/drive/My Drive/CroppedDetection/effNet_model.h5'

# Load the model
model = tf.keras.models.load_model(model_path)

In [ ]:
from google.colab import files

uploaded = files.upload()

# Move the uploaded video to a specific directory
os.makedirs('videos', exist_ok=True)
for file_name in uploaded.keys():
    os.rename(file_name, os.path.join('videos', file_name))

Saving 303.mp4 to 303.mp4


In [ ]:
video_path = 'videos/303.mp4'
new_frames = extract_frames(video_path)
new_frames = np.expand_dims(new_frames, axis=0)
new_frames = np.expand_dims(new_frames, axis=-1)

prediction = model.predict(new_frames)
predicted_class = np.argmax(prediction, axis=1)

print(prediction)
shot = {0: 'Defensive Shot', 1: 'Pull Shot', 2: 'Drive Shot', 3: 'Flick Shot'}
print(shot[predicted_class[0]])

# Defensive | OK     - 253, 254, 300, 129
#           | NOT OK - 234, 211, 84, 137, 303

1/1 [==============================] - 0s 27ms/step
[[0.25894135 0.00087355 0.6204986  0.11968652]]
Drive Shot


In [ ]:
data_dir = '/content/drive/My Drive/ShotDetection'
analysis = check_data(data_dir, model)

1/1 [==============================] - 0s 45ms/step
[[0.01414547 0.00749732 0.9740432  0.00431406]]
1/1 [==============================] - 0s 59ms/step
[[0.96046907 0.00300245 0.02384111 0.01268738]]
1/1 [==============================] - 0s 28ms/step
[[0.8260243  0.00944078 0.15233104 0.01220386]]
1/1 [==============================] - 0s 41ms/step
[[9.6736044e-01 6.1391375e-04 2.2891087e-02 9.1345822e-03]]
1/1 [==============================] - 0s 29ms/step
[[0.0023655  0.9866232  0.00756376 0.00344747]]
1/1 [==============================] - 0s 28ms/step
[[0.25400275 0.51212    0.13280617 0.10107104]]
1/1 [==============================] - 0s 29ms/step
[[0.09673684 0.05245732 0.8434145  0.00739145]]
1/1 [==============================] - 0s 30ms/step
[[0.5560007  0.03899454 0.36489025 0.04011457]]
1/1 [==============================] - 0s 27ms/step
[[0.38021868 0.10596025 0.49111938 0.0227017 ]]
1/1 [==============================] - 0s 32ms/step
[[0.01070444 0.8033917  0.1821369  0

KeyboardInterrupt: 

In [ ]:
analysis

{'Defensive Shot': {'OK': ['45',
   '316',
   '275',
   '47',
   '281',
   '245',
   '260',
   '285',
   '44',
   '306',
   '247',
   '40',
   '239',
   '257',
   '237',
   '43',
   '42',
   '263',
   '68',
   '66',
   '67',
   '15',
   '59',
   '283',
   '56',
   '274',
   '140',
   '235',
   '36',
   '325',
   '9',
   '264',
   '213',
   '280',
   '208',
   '218',
   '219',
   '217',
   '214',
   '172',
   '191',
   '171',
   '187',
   '190',
   '184',
   '188',
   '141',
   '177',
   '161',
   '254',
   '181',
   '147',
   '301',
   '174',
   '154',
   '162',
   '163',
   '284',
   '133',
   '128',
   '304',
   '300',
   '134',
   '135',
   '319',
   '129',
   '91',
   '99',
   '96'],
  'NOT OK': ['318',
   '278',
   '31',
   '236',
   '46',
   '324',
   '276',
   '267',
   '261',
   '241',
   '282',
   '39',
   '243',
   '288',
   '262',
   '41',
   '323',
   '139',
   '64',
   '62',
   '22',
   '16',
   '61',
   '34',
   '238',
   '21',
   '60',
   '55',
   '53',
   '48',
   '57',